## Transorming drivers data and storing it in silver layer for future transformations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
#importing all the required functions
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Defining the Schema for the drivers csv to read the file 

In [0]:
name_struct_schema = StructType([
                                  StructField("forename", StringType(), True),
                                  StructField("surname", StringType(), True)
])

In [0]:
driver_struct_schema = StructType([
                        StructField("driverId", IntegerType(), False),
                        StructField("driverRef", StringType(), True),
                        StructField("number", IntegerType(), True),
                        StructField("code", StringType(), True),
                        StructField("name",name_struct_schema),
                        StructField("dob", DateType(), True),
                        StructField("nationality", StringType(), True),
                        StructField("url", StringType(), True)
])

In [0]:
drivers_df = spark.read.format('json').option('header', True).schema(driver_struct_schema).load(f"{raw_folder_path}/drivers.json")

In [0]:
drivers_final_df = drivers_df.withColumnsRenamed({"driverId": "driver_id", "driverRef" : "driver_ref"})\
    .withColumn("ingestion_date",current_timestamp()).drop(col("url"))\
        .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [0]:
drivers_final_df.write.format('parquet').mode('overwrite').option('path', f"{silver_folder_path}/drivers").save()

In [0]:
 spark.read.format('parquet').option('header', True).option('inferSchema', True).load(f"{silver_folder_path}/drivers").display()